## Imports

In [1]:
import tensorflow as tf
from keras.layers.normalization import layer_normalization

In [2]:
from tensorflow import keras
from keras.layers.normalization import layer_normalization
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import warnings
import numpy as np
# import matplotlib.pyplot as plt
import cv2
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping
import sklearn as sk
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
warnings.simplefilter(action='ignore', category=FutureWarning)

## Train and Test image directory paths

In [3]:
train_path = r'D:\gesture\train'
test_path = r'D:\gesture\test'

In [4]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)

Found 629 images belonging to 3 classes.
Found 413 images belonging to 3 classes.


## Building the model

In [5]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(3,activation ="softmax"))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0005)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

history2 = model.fit(train_batches, epochs=100, callbacks=[reduce_lr, early_stop],  validation_data = test_batches)

Epoch 1/100
63/63 [==============================] - 11s 157ms/step - loss: 1.8390 - accuracy: 0.8426 - val_loss: 0.4546 - val_accuracy: 0.7797
Epoch 2/100
63/63 [==============================] - 3s 41ms/step - loss: 0.0256 - accuracy: 0.9984 - val_loss: 0.2651 - val_accuracy: 0.9274
Epoch 3/100
63/63 [==============================] - 3s 42ms/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 0.2463 - val_accuracy: 0.9225
Epoch 4/100
63/63 [==============================] - 3s 42ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.2431 - val_accuracy: 0.9225
Epoch 5/100
63/63 [==============================] - 2s 37ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.2419 - val_accuracy: 0.9274
Epoch 6/100
63/63 [==============================] - 2s 37ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.2416 - val_accuracy: 0.9249
Epoch 7/100
63/63 [==============================] - 2s 38ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.2425 - val_accuracy: 0.9249
Epoc

In [6]:
history2.history

{'loss': [1.838985562324524,
  0.025573257356882095,
  0.009534615091979504,
  0.005050939042121172,
  0.0035314252600073814,
  0.0026788455434143543,
  0.0021481155417859554,
  0.0018230680143460631],
 'accuracy': [0.842607319355011,
  0.998410165309906,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'val_loss': [0.45458319783210754,
  0.2651228904724121,
  0.24634303152561188,
  0.24308721721172333,
  0.24192912876605988,
  0.2416483759880066,
  0.24252155423164368,
  0.24279923737049103],
 'val_accuracy': [0.7796609997749329,
  0.9273607730865479,
  0.9225181341171265,
  0.9225181341171265,
  0.9273607730865479,
  0.9249394536018372,
  0.9249394536018372,
  0.9249394536018372],
 'lr': [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0005]}

## Saving our model

In [7]:
model.save('signModel')

INFO:tensorflow:Assets written to: signModel\assets


## Metrics

In [10]:
imgs, labels = next(test_batches) 
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))
scores

loss of 0.10377870500087738; accuracy of 100.0%


[0.10377870500087738, 1.0]

In [20]:
imgs, labels = next(test_batches)
model = keras.models.load_model("sign_model")
# new_model = tf.keras.models.load_model('saved_model/my_model')
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))

loss of 0.10839533805847168; accuracy of 89.99999761581421%


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

## Predictions

In [22]:
imgs, labels = next(test_batches) 

word_dict = {0:'One', 1:'Two', 2:'Three'}

predictions = model.predict(imgs, verbose=0)
predicted_labels = []
actual_labels = []

print("Predicted labels")
for ind, i in enumerate(predictions):
    print(word_dict[np.argmax(i)])
    predicted_labels.append(word_dict[np.argmax(i)])
    
print('Actual labels')
for i in labels:
    print(word_dict[np.argmax(i)])
    actual_labels.append(word_dict[np.argmax(i)])


Predicted labels
Two
Two
One
Two
Two
Two
Three
Two
Two
Two
Actual labels
Three
Two
One
Two
Two
Two
Three
Two
Two
Two


In [23]:
print ("Precision", sk.metrics.precision_score(actual_labels, predicted_labels, average='macro'))
print ("Recall", sk.metrics.recall_score(actual_labels, predicted_labels, average='macro'))
print ("f1_score", sk.metrics.f1_score(actual_labels, predicted_labels, average='macro'))
print ("confusion_matrix: ")
print (sk.metrics.confusion_matrix(actual_labels, predicted_labels))

Precision 0.9583333333333334
Recall 0.8333333333333334
f1_score 0.8666666666666666
confusion_matrix: 
[[1 0 0]
 [0 1 1]
 [0 0 7]]


In [ ]:
labels

In [ ]:
predictions